# HVM Creep: Compliance Evolution and Delayed Flow

## Objectives

This tutorial demonstrates fitting the Hybrid Vitrimer Model (HVM) to experimental creep compliance data from polystyrene at 160°C. We use the NLSQ → NumPyro NUTS Bayesian inference pipeline.

**Physical Context:**
- Creep response shows: (1) instantaneous elastic jump from G_P, (2) delayed elastic contribution from E/D networks, (3) steady-state flow where σ_E→0
- HVM exhibits Kelvin-Voigt-like behavior with multiple timescales
- Natural state tracking leads to stress-free steady state (σ_E → 0)

**Workflow:**
1. Load experimental compliance data J(t) = γ(t)/σ₀
2. NLSQ optimization for point estimates
3. Bayesian inference with NUTS for uncertainty quantification
4. Physical interpretation via natural-state evolution
5. Component analysis (elastic vs viscous contributions)

## 1. Setup

In [ ]:
# Float64 configuration (CRITICAL: import before JAX)
from rheojax.core.jax_config import safe_import_jax, verify_float64
jax, jnp = safe_import_jax()
verify_float64()

# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Add utils to path
sys.path.insert(0, str(Path("..").resolve()))

# Model and utilities
from rheojax.models import HVMLocal
from utils.hvm_data import load_ps_creep, check_data_quality
from utils.hvm_fit import (
    run_nlsq_protocol,
    run_nuts,
    print_convergence,
    print_parameter_table,
    posterior_predictive_1d,
    get_bayesian_config,
    get_output_dir,
    save_figure,
    save_results,
    FAST_MODE,
)

print(f"FAST_MODE: {FAST_MODE}")
print(f"JAX version: {jax.__version__}")

## 2. Load Experimental Data

We load creep compliance J(t) = γ(t)/σ₀ data from polystyrene at 160°C (433 K). The data exhibits:
- Instantaneous elastic response (1/G_P)
- Delayed retardation (E/D networks)
- Terminal flow regime (vitrimer bond exchange)

In [ ]:
# Load polystyrene creep data at 160°C
time, J = load_ps_creep(temperature=160, max_points=150)

print(f"Data points: {len(time)}")
print(f"Time range: {time.min():.2e} to {time.max():.2e} s")
print(f"Compliance range: {J.min():.2e} to {J.max():.2e} Pa⁻¹")

# Quality check
check_data_quality(time, J, "PS Creep at 160°C")

## 3. Data Exploration

Visualize the raw compliance data to identify:
- Instantaneous jump J(0) ≈ 1/G_P
- Retardation timescales (E/D networks)
- Terminal slope (viscous flow)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(time, J, 'ko', markersize=6, label='PS 160°C')
ax.set_xlabel('Time t (s)', fontsize=12)
ax.set_ylabel('Compliance J(t) (Pa⁻¹)', fontsize=12)
ax.set_title('Polystyrene Creep Compliance: Raw Data', fontsize=13, weight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3, which='both', ls='-', lw=0.5)
plt.tight_layout()
display(fig)
plt.close(fig)

## 4. Model Setup

Initialize HVMLocal with parameters appropriate for polystyrene at 160°C (433 K).

**Key parameters for creep:**
- **G_P**: Permanent network modulus (instantaneous response)
- **G_E**: Exchangeable network modulus (delayed elasticity)
- **G_D**: Dissociative network modulus (retardation)
- **k_d_D**: Dissociation rate (controls terminal flow)
- **T**: Temperature (433 K = 160°C)

In [ ]:
# Initialize model with dissociative network
model = HVMLocal(include_dissociative=True, kinetics='stress')

# Set initial parameter values for PS at 160°C
model.parameters.set_value("G_P", 50000.0)   # Permanent network
model.parameters.set_value("G_E", 30000.0)   # Exchangeable network
model.parameters.set_value("G_D", 10000.0)   # Dissociative network
model.parameters.set_value("nu_0", 1e12)     # Exchange frequency (higher for PS)
model.parameters.set_value("E_a", 100e3)     # Activation energy (typical for PS)
model.parameters.set_value("V_act", 1e-5)   # Activation volume (m³)
model.parameters.set_value("k_d_D", 0.1)     # Dissociation rate
model.parameters.set_value("T", 433.0)       # 160°C in Kelvin

print("Model initialized with parameters:")
print(model.parameters)

## 5. NLSQ Fit

Perform non-linear least squares optimization using NLSQ.

**Important Notes:**
- Data is already compliance J(t) = γ(t)/σ₀
- Use `sigma_applied=1.0` since data is normalized
- Model will predict γ(t) and convert to J(t) internally

In [ ]:
# Run NLSQ optimization
print("Running NLSQ optimization for creep...")
nlsq_vals = run_nlsq_protocol(
    model,
    time,
    J,
    test_mode='creep',
    sigma_applied=1.0,  # Data is normalized compliance
    use_log_residuals=False  # Linear residuals for compliance
)

print("\nNLSQ optimization complete.")
print(f"Fitted parameters: {nlsq_vals}")
print(f"\nModel R²: {(model._nlsq_result.r_squared or 0):.4f}")

## 6. Plot NLSQ Fit

Visualize the fitted compliance curve and compare to data.

In [ ]:
# Generate dense prediction grid
time_fit = np.logspace(np.log10(time.min()), np.log10(time.max()), 200)

# Get compliance prediction (J = γ/σ₀)
gamma_fit = model.simulate_creep(time_fit, sigma_0=1.0)
J_fit = gamma_fit  # Since σ₀ = 1.0

# Plot
fig, ax = plt.subplots(figsize=(9, 6))

# Data
ax.loglog(time, J, 'ko', markersize=7, label='Data (PS 160°C)', zorder=3)

# Fit
ax.loglog(time_fit, J_fit, 'r-', lw=2.5, label='HVM Fit', zorder=2)

ax.set_xlabel('Time t (s)', fontsize=12)
ax.set_ylabel('Compliance J(t) (Pa⁻¹)', fontsize=12)
ax.set_title('HVM Creep Compliance: NLSQ Fit', fontsize=13, weight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3, which='both', ls='-', lw=0.5)
plt.tight_layout()
display(fig)
plt.close(fig)

# Save figure
save_figure(fig, get_output_dir('creep'), 'hvm_creep_nlsq_fit.png')

## 7. Bayesian Inference with NUTS

Perform Bayesian inference using NumPyro NUTS sampler with NLSQ warm-start.

**Configuration:**
- 4 chains for production-grade diagnostics
- NLSQ values as warm-start
- ArviZ diagnostics for convergence assessment

In [ ]:
# Get Bayesian configuration (respects FAST_MODE)
bayes_config = get_bayesian_config()
print(f"Bayesian config: {bayes_config}")

# Run NUTS
print("\nRunning Bayesian inference (NUTS)...")
result = run_nuts(
    model,
    time,
    J,
    test_mode='creep',
    **bayes_config
)

# Define parameter names for diagnostics
param_names = list(model.parameters.keys())

print("\nBayesian inference complete.")

### 7.1 Convergence Diagnostics

In [ ]:
# Print convergence diagnostics
print_convergence(result, param_names)

# Print parameter summary table
print("\n" + "="*60)
print("PARAMETER POSTERIOR SUMMARY")
print("="*60)
print_parameter_table(param_names, nlsq_vals, result.posterior_samples)

## 8. Posterior Predictive Analysis

Generate posterior predictive distribution to quantify uncertainty in compliance predictions.

In [ ]:
# Generate posterior predictive samples
print("Generating posterior predictive distribution...")
pred_draws = posterior_predictive_1d(
    model,
    time_fit,
    result.posterior_samples,
    test_mode='creep',
    n_draws=100
)

# Compute statistics
pred_mean = np.mean(pred_draws, axis=0)
pred_lo = np.percentile(pred_draws, 2.5, axis=0)
pred_hi = np.percentile(pred_draws, 97.5, axis=0)

# Plot posterior predictive
fig, ax = plt.subplots(figsize=(9, 6))

# Uncertainty bands (95% credible interval)
ax.fill_between(time_fit, pred_lo, pred_hi, color='blue', alpha=0.15, label='95% CI')
ax.loglog(time_fit, pred_mean, 'b-', lw=2, label='Posterior mean')

# Data
ax.loglog(time, J, 'ko', markersize=5, alpha=0.6, label='Data')

ax.set_xlabel("Time (s)", fontsize=12)
ax.set_ylabel("Compliance J(t) (1/Pa)", fontsize=12)
ax.set_title("HVM Creep: Posterior Predictive", fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
save_figure(fig, get_output_dir('creep'), 'hvm_creep_posterior_predictive.png')
plt.close(fig)

## 9. Physical Interpretation: Natural State Evolution

Run full simulation to extract internal state variables and demonstrate:
- Evolution of μ^E_nat (natural state tracking)
- Stress relaxation in exchangeable network (σ_E → 0)
- Multi-timescale retardation from E/D networks

In [ ]:
# Run full creep simulation
print("Running full creep simulation...")
J_fit = model.predict(time_fit, test_mode='creep')

# Plot creep prediction
fig, ax = plt.subplots(figsize=(9, 6))

# Data
ax.loglog(time, J, 'ko', markersize=4, alpha=0.5, label='PS creep data')

# Fit
ax.loglog(time_fit, J_fit, 'r-', lw=2.5, label='HVM NLSQ fit')

ax.set_xlabel("Time (s)", fontsize=12)
ax.set_ylabel("Compliance J(t) (1/Pa)", fontsize=12)
ax.set_title("HVM Creep: NLSQ Fit (Detailed)", fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
save_figure(fig, get_output_dir('creep'), 'hvm_creep_internal_states.png')
plt.close(fig)

### Physical Insights from Natural State Evolution

**Panel (b)** shows the hallmark of vitrimer behavior:
- Initially: μ^E_nat,xy = 0 (unstrained natural state)
- Over time: μ^E_nat,xy → μ^E_xy (natural state tracks deformation)
- Result: σ_E = G_E(μ^E - μ^E_nat) → 0 (stress-free steady state)

**Panel (c)** confirms stress relaxation:
- E-network stress decays with timescale τ_E = 1/(2·k_BER)
- Factor-of-2 arises from mutual relaxation of μ^E and μ^E_nat

**Panel (d)** reveals retardation spectrum:
- Multiple peaks correspond to E/D network timescales
- Terminal slope reflects viscous flow from D-network

## 10. Component Analysis: Elastic vs Viscous Contributions

Decompose compliance into:
- **Instantaneous elastic**: 1/G_P
- **Delayed elastic**: E/D network retardation
- **Viscous flow**: Terminal slope

In [ ]:
# Extract fitted parameters
G_P = model.parameters.get_value('G_P')
G_E = model.parameters.get_value('G_E')
G_D = model.parameters.get_value('G_D')
k_d_D = model.parameters.get_value('k_d_D')

# Instantaneous compliance
J_0 = 1.0 / G_P

# Terminal compliance slope (from D-network viscosity)
eta_D = G_D / k_d_D
J_terminal = J_0 + time_fit / eta_D

# Plot decomposition
fig, ax = plt.subplots(figsize=(9, 6))

# Data and total fit
ax.loglog(time, J, 'ko', markersize=7, label='Data', zorder=3)
ax.loglog(time_fit, J_fit, 'r-', lw=2.5, label='HVM Total', zorder=2)

# Instantaneous elastic
ax.axhline(J_0, color='blue', ls='--', lw=2, alpha=0.7, label=f'J₀ = 1/G_P = {J_0:.2e} Pa⁻¹')

# Terminal viscous flow
ax.loglog(time_fit, J_terminal, 'g-.', lw=2, alpha=0.7, label='J₀ + t/η_D (terminal)')

ax.set_xlabel('Time t (s)', fontsize=12)
ax.set_ylabel('Compliance J(t) (Pa⁻¹)', fontsize=12)
ax.set_title('HVM Creep: Component Decomposition', fontsize=13, weight='bold')
ax.legend(loc='best', fontsize=9)
ax.grid(True, alpha=0.3, which='both', ls='-', lw=0.5)
plt.tight_layout()
display(fig)
plt.close(fig)

# Save figure
save_figure(fig, get_output_dir('creep'), 'hvm_creep_component_decomposition.png')

## 11. Save Results

Save all results for reproducibility:
- NLSQ parameters
- Posterior samples (HDF5)
- Figures (PNG)
- Metadata (config, diagnostics)

In [ ]:
# Save results
output_dir = get_output_dir('creep')
print(f"Saving results to: {output_dir}")

save_results(output_dir, model=model, result=result, param_names=param_names)

print("Results saved successfully.")

## Summary

**Key Findings:**
1. HVM successfully captures multi-timescale creep response in polystyrene
2. Instantaneous compliance 1/G_P from permanent network
3. Delayed retardation from E/D networks with distinct timescales
4. Terminal viscous flow from D-network (η_D = G_D/k_d_D)
5. Bayesian inference provides robust uncertainty quantification

**Physical Insights:**
- Natural state tracking: μ^E_nat → μ^E over timescale τ_E = 1/(2·k_BER)
- E-network stress relaxation: σ_E → 0 at steady state (stress-free)
- Factor-of-2 in effective timescale from mutual relaxation
- Vitrimer bond exchange enables stress-free steady flow

**Extracted Parameters:**
- G_P: Permanent network modulus (elastic plateau)
- G_E: Exchangeable network modulus (vitrimer)
- G_D: Dissociative network modulus (retardation)
- k_d_D: Dissociation rate (terminal viscosity)
- ν₀, E_a: Arrhenius parameters for BER kinetics

**Next Steps:**
- Temperature sweep to extract Arrhenius parameters
- Compare to startup/relaxation for consistency
- Investigate stress-dependence via TST kinetics
- Explore parameter correlations via pair plots